In [ ]:
"""
Author: Chenchen Gu (cygu@stanford.edu)
CS 109 Challenge, Winter 2022
"""

In [1]:
import music21
from tqdm import tqdm
import numpy as np

In [2]:
def transpose_part(part, key):
    """Transposes a part to the key of C major."""
    transpose_interval = music21.interval.Interval(key.tonic, music21.pitch.Pitch('C'))
    return part.transpose(transpose_interval)

In [3]:
def count_midi_frequencies(music, midi_freq_list):
    """
    Count the frequencies of transitions between MIDI pitches in `music`, and update 
    `midi_freq_list` with those counts. `midi_freq_list` should be a list of lists,
    where `midi_freq_list` is of length 128, and so are all of the lists at each of
    its indexes. Each index value represents the corresponding MIDI value.
    """
    for part in music.parts():
        key = part.recurse().getElementsByClass('Key')[0]
        # Only analyze pieces in major mode
        if key.mode != 'major':
            continue

        # Transpose all notes to the key of C
        transpose_interval = music21.interval.Interval(key.tonic, music21.pitch.Pitch('C'))
        prev_midi = 0

        for elem in part.flatten().getElementsByClass(['Note', 'Rest', 'Key', 'KeySignature']):
            if key.mode != 'major':
                continue
                
            cur_midi = 0
            if isinstance(elem, music21.key.Key):
                key = elem
            if isinstance(elem, music21.key.KeySignature):
                key = elem.asKey()
                
            if isinstance(elem, music21.note.Note):
                cur_midi = elem.transpose(transpose_interval).pitch.midi

            if prev_midi != 0 and cur_midi != 0:
                midi_freq_list[prev_midi][cur_midi] += 1

            prev_midi = cur_midi
    
    return midi_freq_list

In [4]:
def count_rhythm_frequencies(music, rhythm_freq_dict):
    """
    Count the frequencies of rhythmic durations, given the sub-beat that it occurs on.
    The sub-beat is the decimal part of the beat (e.g., sub-beat of 2.5 is 0.5).
    rhythm_freq_dict is a dictionary whose keys are sub-beats and whoses values are
    nested dict. Each nested dict maps rhythmic durations, in quarter lengths, to frequency
    counts.
    """
    for part in music.parts:
        for note in part.flatten().notes:
            sub_beat = note.beat % 1.0
            length = note.quarterLength
            
            # Remove tuplets because floating point (1/3) will likely lead to issues
            if sub_beat % 0.25 != 0 or length % 0.25 != 0:
                continue
            
            if sub_beat not in rhythm_freq_dict:
                rhythm_freq_dict[sub_beat] = {}
                
            if length not in rhythm_freq_dict[sub_beat]:
                rhythm_freq_dict[sub_beat][length] = 0
                
            rhythm_freq_dict[sub_beat][length] += 1
    
    return rhythm_freq_dict

In [5]:
midi_freq_list = [ ([0] * 128) for _ in range(128) ]
rhythm_freq_dict = {}

In [ ]:
with open("kernscores_urls.txt") as file:
    url_list = file.read().splitlines()

for url in tqdm(url_list):
    try:
        music = music21.converter.parse(url)
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except:
        print("Invalid url:", url)
        continue
    count_midi_frequencies(music, midi_freq_list)
    count_rhythm_frequencies(music, rhythm_freq_dict)

In [ ]:
print(rhythm_freq_dict)

In [ ]:
import json

# Save frequencies data to json file
with open('../data/midi_frequencies.json', 'w') as file:
    #json.dump(midi_freq_list, file)

with open('../data/rhythm_frequencies.json', 'w') as file:
    #json.dump(rhythm_freq_dict, file)